<a href="https://colab.research.google.com/github/saharaty/LLM/blob/main/sampleLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Installment

In [ ]:
!pip install torch

##Code

In [ ]:
import time
import numpy as np
import torch.nn as nn
from torch.nn import functional as F
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [ ]:
block_size = 8 #size of the integers
batch_size = 4 #number ofprocessing sequences in parallel
max_iters = 1000
eval_interval = 500 #interval for model evaluation
learning_rate = 3e-4
eval_iters = 250
#dropout = 0.2 #to dropout random neurons to not to overfit

In [ ]:
#load the dataset
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
  text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [ ]:
#map characters and integers
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

#convert the encoded text to a tensor
data = torch.tensor(encode(text), dtype=torch.long)

In [ ]:
#making training and validation sets
n = int(0.8 * len(data))
trian_set = data[:n]
val_set = data[n:]

#generate batches
def get_batch(split):
  data = trian_set if split == 'train' else val_set
  #choose random starting indices
  ix = torch.randint(len(data) - block_size, (batch_size,))

  x = torch.stack([data[i:i+block_size] for i in ix]) #input sequence
  y = torch.stack([data[i+1:i+block_size+1] for i in ix]) #target sequence
  x, y = x.to(device), y.to(device) #move data to appropriate device
  return x, y

x, y = get_batch("train")


In [ ]:
#estimate loss on training and validaion sets
@torch.no_grad() #use this to not to use gradient on validation set, for better performance
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean() #calculate the average loss
  model.train()
  return out

In [ ]:
class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) #map tokens to vectors

  #the forward pass
  def forward(self, index, targets=None):
    logits = self.token_embedding_table(index) #pass the index through the embedding layer to map the index to a vector

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape  #B batch size, T seq length, C vocabulary size
      logits = logits.view(B*T, C)  #reshape logits, to compute loss
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss


  def generate(self, index, max_new_tokens):
    for _ in range(max_new_tokens): #index is (B, T)
      logits, loss = self.forward(index)  #prediction
      logits = logits[:, -1, :] #focus on the last time step
      probs = F.softmax(logits, dim=-1) #apply softmax to get probabilities
      index_next = torch.multinomial(probs, num_samples=1)  #sampling
      index = torch.cat((index, index_next), dim=1) #append sample (B, T+1)
    return index


model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long , device=device) #torch.long : int 64
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


x_wcXzw!(Deid*.T(DqHydqa_b2l8[Mi9TJzJFkgyAFGwl;SVV?﻿uD1wxUy"7mDmR6xc]bTp]1mvxB(kkOiNY_4.9rZl:Q﻿2[st6jFsUjWsXQqzZXnR0vejvGw!EA15]jvJ2jD8.D9c]Cv6 t
*Z:x
0cZPB4b G?f-1 tu)]ztZjDy2Jnz"ep;H'my?2LC_wDTdD9VuzuN0h8JQzNIS﻿V_T,H9l?bTgHG?T"Bt3*tu4.g
4J5l)e]u?I7gYQe,dLL8L5X2nOi"AN,fQ﻿HmRa6nb*]!cv3D_yW
;H,qrZP&gsXPF*]kddrV]Ow4FDt_15Nh'm.9yK]Srgm.﻿Lrgma*7 'yOQ]'VQG4YHtR['f"6Nzz&2lS:Gt3,V]HigH﻿bp'11u-rS[HjaK)rC4D9TQ".b96t_uCAfa_E z-R0VY0Gm;zLE3j3:QvqCAEG*LZTw)﻿9﻿52 jxhwz6PNOrkwdyZ*BbaIL8J5qM167RNohqaHf]dP mPxi


In [ ]:
#Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for i in range(max_iters):
  if i % eval_iters == 0:
    losses = estimate_loss() #evaluate the loss on both training and validation set
    print(f"step {i}, train loss {losses['train']}, validation loss: {losses['val']}")

  xb, yb = get_batch('train') #get a batch of training data

  logits, loss = model.forward(xb, yb)
  optimizer.zero_grad(set_to_none=True) #reset the gradients
  loss.backward() #calculate the gradients
  optimizer.step() #update parameters
print(loss.item())

step 0, train loss 3.4473421573638916, validation loss: 3.453442096710205
step 250, train loss 3.4152681827545166, validation loss: 3.428668737411499
step 500, train loss 3.36126971244812, validation loss: 3.422299861907959
step 750, train loss 3.3270020484924316, validation loss: 3.346626043319702
3.6360726356506348


In [ ]:
#sample process
x = trian_set[:block_size] #input
y = trian_set[1:block_size+1] #target

 #it just knows current char
for i in range(block_size):
  context = x[: i+1]
  target = y[i]
  print('when input is', context, 'target is', target)

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(1)
when input is tensor([80,  1,  1]) target is tensor(28)
when input is tensor([80,  1,  1, 28]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [ ]:
#generate texe
context = torch.zeros((1,1), dtype=torch.long , device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


N&RHjayjYUa!CGep!)l54A;L!jtjY*XQWlr1,M0(JVR&KevO0_zD:]z﻿8wBY_F8'-xCWfWdXZm:6xkhCsJVafY95T4﻿c0UfkNfeO0nFuZPg.MMH2yM&sFD(mLki:
NI3uN)00q,.)y;dl﻿P:]zg;?8kM﻿)w:eydOa;g6,NE﻿4q?wNE86"IBoMK4&'-[]ULxRHs﻿rP;nd!3kj2G7JX*h*HT54R-&YV?﻿wtGR]2TP2q?RSNjWK('mV(z['a!VXMHFR1lmtANL!CDPheNDM!f1sFx.)"JUT4CWaAg514ypd(Wlmy aq*S:1s7'!J6Cky73Pe3my!HbJ1s,.)GE6dqp!jAFzDg50bkGMBlx])0R2 v:0y'mIPM[4﻿g7]VM0RG8IEP:]PVgCKp-I
fz)Wlx7J15&RWa;F
&"c'MhPn!';i9(noHlw
Zugwe80WCyOzjALVXD!
2 o'm4]B]RiUgwRr-SP-K_VR]1b;
;miCOm1Lw7slq9"0P1
